In [1]:
#temp
import numpy as np
import mediapipe as mp
import joblib
from sklearn.preprocessing import LabelEncoder
import tkinter as tk
from PIL import Image, ImageTk
import os
import time
from datetime import datetime
import cv2
import azure.cognitiveservices.speech as speechsdk

# === Azure Speech API credentials ===
speech_key = "C2aQwIVI4DwKew11iZqZiOn4x1FEt7qgaM2qIfDZIdCXnZm9LEfMJQQJ99BEACYeBjFXJ3w3AAAEACOGfGDZ"
service_region = "eastus"

# Azure configuration
speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
speech_config.speech_synthesis_voice_name = "sw-KE-ZuriNeural"
synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config)

# === Load ML Model and Encode Labels ===
model = joblib.load('mlp_tsl_static.pkl')
le = LabelEncoder()
le.fit([chr(i) for i in range(ord('A'), ord('Z') + 1)])

# === MediaPipe Hand Model ===
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1,
                       min_detection_confidence=0.7, min_tracking_confidence=0.5)

# === Normalize hand landmarks ===
def normalize_landmarks(landmarks):
    coords = np.array(landmarks).reshape(-1, 3).astype(np.float32)
    coords_min = coords.min(axis=0)
    coords_max = coords.max(axis=0)
    norm_coords = (coords - coords_min) / (coords_max - coords_min + 1e-6)
    return norm_coords.flatten().reshape(1, -1)

# === GUI Application Class ===
class TSLApp:
    def __init__(self, root):
        self.root = root
        self.root.title("TSL - Bridging Silence")
        self.root.configure(bg="#f0f4f8")

        self.video_running = False
        self.cap = None

        self.prev_letter = ""
        self.letter_hold_start = None
        self.last_seen_time = time.time()
        self.word = ""
        self.sentence = ""
        self.saved_sentences = []

        # === Title ===
        self.title_label = tk.Label(root, text="BRIDGING SILENCE", font=("Arial", 24, "bold"),
                                    fg="#005073", bg="#f0f4f8")
        self.title_label.pack(pady=10)

        # === Horizontal Layout ===
        self.container = tk.Frame(root, bg="#f0f4f8")
        self.container.pack(fill="both", expand=True)

        # === Left Panel: Camera Feed & Predictions ===
        self.left_panel = tk.Frame(self.container, bg="#f0f4f8")
        self.left_panel.pack(side="left", padx=10, pady=10)

        self.video_label = tk.Label(self.left_panel, bg="#e6ecf0", bd=2, relief="solid")
        self.video_label.pack()

        self.prediction_label = tk.Label(
            self.left_panel,
            text="Letter: \nWord: \nSentence:",
            font=("Arial", 16),
            fg="#005073",
            bg="#f0f4f8",
            justify="left",
            anchor="w",
            padx=10,
            pady=10
        )
        self.prediction_label.pack(pady=10, anchor="w")

        # === Right Panel: Control Buttons ===
        self.controls = tk.Frame(self.container, bg="#f0f4f8")
        self.controls.pack(side="right", padx=20, pady=10, fill="y")

        button_style = {"font": ("Arial", 12), "width": 12, "padx": 5, "pady": 5}

        tk.Button(self.controls, text="Start", command=self.start_video,
                  bg="#28a745", fg="white", **button_style).pack(pady=4)
        tk.Button(self.controls, text="Stop", command=self.stop_video,
                  bg="#dc3545", fg="white", **button_style).pack(pady=4)
        tk.Button(self.controls, text="Clear", command=self.clear_predictions,
                  bg="#ffc107", **button_style).pack(pady=4)
        tk.Button(self.controls, text="Speak", command=self.speak_text,
                  bg="#17a2b8", fg="white", **button_style).pack(pady=4)
        tk.Button(self.controls, text="Del Letter", command=self.delete_last_letter,
                  bg="#6c757d", fg="white", **button_style).pack(pady=4)
        tk.Button(self.controls, text="Del Word", command=self.delete_last_word,
                  bg="#343a40", fg="white", **button_style).pack(pady=4)

    # === Button Functionalities ===

    def start_video(self):
        if not self.video_running:
            self.cap = cv2.VideoCapture(0)
            self.video_running = True
            self.update_video()

    def stop_video(self):
        self.video_running = False
        if self.cap:
            self.cap.release()
        self.video_label.config(image='')

    def clear_predictions(self):
        self.word = ""
        self.sentence = ""
        self.saved_sentences.clear()
        self.prediction_label.config(text="Letter: \nWord: \nSentence:")

    def speak_text(self):
        full_sentence = (self.sentence + self.word).strip()
        if full_sentence:
            try:
                result = synthesizer.speak_text_async(full_sentence).get()
                if result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
                    print("Speech synthesis completed.")
                else:
                    print("Speech synthesis failed:", result.reason)
            except Exception as e:
                print("Azure speech error:", e)

            self.saved_sentences.append(full_sentence)
            self.word = ""
            self.sentence = ""
            self.prediction_label.config(text="Letter: \nWord: \nSentence:")

    def delete_last_letter(self):
        if self.word:
            self.word = self.word[:-1]
        elif self.sentence:
            self.sentence = self.sentence.rstrip()
            if self.sentence and self.sentence[-1] == " ":
                self.sentence = self.sentence[:-1]
            self.word = self.sentence.split()[-1] if self.sentence else ""
            self.sentence = " ".join(self.sentence.split()[:-1]) + " "
        self.prediction_label.config(text=f"Letter: \nWord: {self.word}\nSentence: {self.sentence}")

    def delete_last_word(self):
        if self.word:
            self.word = ""
        elif self.sentence:
            self.sentence = self.sentence.rstrip()
            words = self.sentence.split()
            self.sentence = " ".join(words[:-1]) + " " if words else ""
        self.prediction_label.config(text=f"Letter: \nWord: {self.word}\nSentence: {self.sentence}")

    # === Update video feed and predict signs ===
    def update_video(self):
        if not self.video_running:
            return

        ret, frame = self.cap.read()
        if not ret:
            self.stop_video()
            return

        frame = cv2.flip(frame, 1)
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb)

        current_time = time.time()
        current_letter = ""

        if results.multi_hand_landmarks:
            self.last_seen_time = current_time
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                landmarks = [(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark]

                try:
                    X = normalize_landmarks(landmarks)
                    pred_index = model.predict(X)[0]
                    current_letter = le.inverse_transform([pred_index])[0]

                    if current_letter == self.prev_letter:
                        if not self.letter_hold_start:
                            self.letter_hold_start = current_time
                        if current_time - self.letter_hold_start >= 1:
                            if not self.word or self.word[-1] != current_letter:
                                self.word += current_letter
                    else:
                        self.letter_hold_start = current_time

                    self.prev_letter = current_letter

                except Exception as e:
                    print("Prediction error:", e)
        else:
            time_since_last = current_time - self.last_seen_time
            if time_since_last >= 2 and self.word and (not self.word.endswith(" ")):
                self.word += " "
            if time_since_last >= 5 and self.word.strip():
                self.sentence += self.word.strip() + " "
                self.word = ""

        display_text = f"Letter: {current_letter}\nWord: {self.word}\nSentence: {self.sentence}"
        self.prediction_label.config(text=display_text)

        img = Image.fromarray(rgb)
        imgtk = ImageTk.PhotoImage(image=img)
        self.video_label.imgtk = imgtk
        self.video_label.configure(image=imgtk)

        self.root.after(10, self.update_video)

# === Run the Application ===
if __name__ == "__main__":
    root = tk.Tk()
    app = TSLApp(root)
    root.mainloop()


d:\WIN AND LEARN\.venv\Lib\site-packages\sklearn\base.py:440: InconsistentVersionWarning: Trying to unpickle estimator LabelBinarizer from version 1.6.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
d:\WIN AND LEARN\.venv\Lib\site-packages\sklearn\base.py:440: InconsistentVersionWarning: Trying to unpickle estimator MLPClassifier from version 1.6.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [2]:

import numpy as np
import mediapipe as mp
import joblib
from sklearn.preprocessing import LabelEncoder
import tkinter as tk
from tkinter import ttk, messagebox
from PIL import Image, ImageTk, ImageDraw, ImageFont
import os
import time
from datetime import datetime
import cv2
import azure.cognitiveservices.speech as speechsdk

# === Azure Speech API credentials ===
speech_key = "C2aQwIVI4DwKew11iZqZiOn4x1FEt7qgaM2qIfDZIdCXnZm9LEfMJQQJ99BEACYeBjFXJ3w3AAAEACOGfGDZ"
service_region = "eastus"

# Azure configuration
speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
speech_config.speech_synthesis_voice_name = "sw-KE-ZuriNeural"
synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config)

# === Load ML Model and Encode Labels ===
try:
    model = joblib.load('mlp_tsl_static.pkl')
    le = LabelEncoder()
    le.fit([chr(i) for i in range(ord('A'), ord('Z') + 1)])
except FileNotFoundError:
    model = None
    le = None
    print("Model file not found. Running in demo mode.")

# === MediaPipe Hand Model ===
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1,
                       min_detection_confidence=0.7, min_tracking_confidence=0.5)

# === Color Scheme ===
COLORS = {
    'primary': '#2E86AB',
    'secondary': '#A23B72',
    'accent': '#F18F01',
    'success': '#4CAF50',
    'warning': '#FF9800',
    'danger': '#F44336',
    'background': '#F5F7FA',
    'surface': '#FFFFFF',
    'text_primary': '#2C3E50',
    'text_secondary': '#7B8394',
    'border': '#E1E8ED'
}

# === Normalize hand landmarks ===
def normalize_landmarks(landmarks):
    coords = np.array(landmarks).reshape(-1, 3).astype(np.float32)
    coords_min = coords.min(axis=0)
    coords_max = coords.max(axis=0)
    norm_coords = (coords - coords_min) / (coords_max - coords_min + 1e-6)
    return norm_coords.flatten().reshape(1, -1)

# === Modern GUI Application Class ===
class ModernTSLApp:
    def __init__(self, root):
        self.root = root
        self.root.title("TSL - Bridging Silence")
        self.root.configure(bg=COLORS['background'])
        self.root.geometry("1200x800")
        self.root.minsize(1000, 600)

        # Configure styles
        self.setup_styles()

        # Initialize variables
        self.video_running = False
        self.cap = None
        self.prev_letter = ""
        self.letter_hold_start = None
        self.last_seen_time = time.time()
        self.word = ""
        self.sentence = ""
        self.saved_sentences = []
        self.confidence_score = 0.0

        # Create the interface
        self.create_header()
        self.create_main_content()
        self.create_status_bar()

        # Bind keyboard shortcuts
        self.root.bind('<Control-s>', lambda e: self.start_video())
        self.root.bind('<Control-q>', lambda e: self.stop_video())
        self.root.bind('<Control-c>', lambda e: self.clear_predictions())
        self.root.bind('<Control-space>', lambda e: self.speak_text())
        self.root.bind('<BackSpace>', lambda e: self.delete_last_letter())
        self.root.bind('<Control-BackSpace>', lambda e: self.delete_last_word())

    def setup_styles(self):
        """Configure modern styling for tkinter widgets"""
        self.style = ttk.Style()
        self.style.theme_use('clam')
        
        # Configure button styles
        self.style.configure('Primary.TButton',
                           background=COLORS['primary'],
                           foreground='white',
                           borderwidth=0,
                           focuscolor='none',
                           padding=(20, 10))
        
        self.style.configure('Success.TButton',
                           background=COLORS['success'],
                           foreground='white',
                           borderwidth=0,
                           focuscolor='none',
                           padding=(20, 10))
        
        self.style.configure('Warning.TButton',
                           background=COLORS['warning'],
                           foreground='white',
                           borderwidth=0,
                           focuscolor='none',
                           padding=(20, 10))
        
        self.style.configure('Danger.TButton',
                           background=COLORS['danger'],
                           foreground='white',
                           borderwidth=0,
                           focuscolor='none',
                           padding=(20, 10))

    def create_header(self):
        """Create the header with title and status indicators"""
        header_frame = tk.Frame(self.root, bg=COLORS['surface'], height=80)
        header_frame.pack(fill='x', padx=20, pady=(20, 0))
        header_frame.pack_propagate(False)

        # Title
        title_label = tk.Label(header_frame, 
                              text="🤝 BRIDGING SILENCE", 
                              font=('Arial', 28, 'bold'),
                              fg=COLORS['primary'], 
                              bg=COLORS['surface'])
        title_label.pack(side='left', pady=20)

        # Status indicators
        status_frame = tk.Frame(header_frame, bg=COLORS['surface'])
        status_frame.pack(side='right', pady=20)

        self.camera_status = tk.Label(status_frame, 
                                     text="● Camera: OFF", 
                                     font=('Arial', 12),
                                     fg=COLORS['danger'], 
                                     bg=COLORS['surface'])
        self.camera_status.pack(pady=2)

        self.model_status = tk.Label(status_frame, 
                                    text="● Model: READY" if model else "● Model: NOT FOUND", 
                                    font=('Arial', 12),
                                    fg=COLORS['success'] if model else COLORS['danger'], 
                                    bg=COLORS['surface'])
        self.model_status.pack(pady=2)

    def create_main_content(self):
        """Create the main content area with video feed and controls"""
        main_frame = tk.Frame(self.root, bg=COLORS['background'])
        main_frame.pack(fill='both', expand=True, padx=20, pady=20)

        # Left panel - Video feed
        left_panel = tk.Frame(main_frame, bg=COLORS['surface'], relief='flat', bd=2)
        left_panel.pack(side='left', fill='both', expand=True, padx=(0, 10))

        # Video frame with modern styling
        video_frame = tk.Frame(left_panel, bg=COLORS['surface'])
        video_frame.pack(fill='both', expand=True, padx=20, pady=20)

        video_label_frame = tk.Frame(video_frame, bg=COLORS['border'], relief='flat', bd=2)
        video_label_frame.pack(fill='both', expand=True)

        self.video_label = tk.Label(video_label_frame, 
                                   text="📹 Camera Feed\n\nPress 'Start Camera' to begin",
                                   font=('Arial', 14),
                                   fg=COLORS['text_secondary'],
                                   bg=COLORS['background'],
                                   compound='center')
        self.video_label.pack(fill='both', expand=True)

        # Prediction display
        pred_frame = tk.Frame(left_panel, bg=COLORS['surface'])
        pred_frame.pack(fill='x', padx=20, pady=(0, 20))

        # Current letter display
        letter_frame = tk.Frame(pred_frame, bg=COLORS['accent'], relief='flat', bd=0)
        letter_frame.pack(fill='x', pady=(0, 10))

        tk.Label(letter_frame, text="Current Letter", 
                font=('Arial', 12, 'bold'), 
                fg='white', bg=COLORS['accent']).pack(pady=5)

        self.letter_display = tk.Label(letter_frame, 
                                      text="—", 
                                      font=('Arial', 36, 'bold'),
                                      fg='white', 
                                      bg=COLORS['accent'])
        self.letter_display.pack(pady=10)

        # Confidence indicator
        self.confidence_frame = tk.Frame(pred_frame, bg=COLORS['surface'])
        self.confidence_frame.pack(fill='x', pady=(0, 10))

        tk.Label(self.confidence_frame, text="Confidence", 
                font=('Arial', 10), 
                fg=COLORS['text_secondary'], 
                bg=COLORS['surface']).pack(anchor='w')

        self.confidence_var = tk.DoubleVar()
        self.confidence_bar = ttk.Progressbar(self.confidence_frame, 
                                             variable=self.confidence_var,
                                             maximum=100,
                                             style='TProgressbar')
        self.confidence_bar.pack(fill='x', pady=2)

        # Right panel - Controls and text output
        right_panel = tk.Frame(main_frame, bg=COLORS['surface'], relief='flat', bd=2)
        right_panel.pack(side='right', fill='y', padx=(10, 0))
        right_panel.configure(width=400)
        right_panel.pack_propagate(False)

        # Control buttons
        controls_frame = tk.Frame(right_panel, bg=COLORS['surface'])
        controls_frame.pack(fill='x', padx=20, pady=20)

        tk.Label(controls_frame, text="Controls", 
                font=('Arial', 16, 'bold'), 
                fg=COLORS['text_primary'], 
                bg=COLORS['surface']).pack(anchor='w', pady=(0, 10))

        # Button grid
        button_frame = tk.Frame(controls_frame, bg=COLORS['surface'])
        button_frame.pack(fill='x')

        self.start_btn = ttk.Button(button_frame, text="▶ Start Camera", 
                                   command=self.start_video, style='Success.TButton')
        self.start_btn.pack(fill='x', pady=2)

        self.stop_btn = ttk.Button(button_frame, text="⏹ Stop Camera", 
                                  command=self.stop_video, style='Danger.TButton')
        self.stop_btn.pack(fill='x', pady=2)

        ttk.Button(button_frame, text="🗑 Clear All", 
                  command=self.clear_predictions, style='Warning.TButton').pack(fill='x', pady=2)

        ttk.Button(button_frame, text="🔊 Speak Text", 
                  command=self.speak_text, style='Primary.TButton').pack(fill='x', pady=2)

        # Edit buttons
        edit_frame = tk.Frame(button_frame, bg=COLORS['surface'])
        edit_frame.pack(fill='x', pady=(10, 0))

        ttk.Button(edit_frame, text="⌫ Delete Letter", 
                  command=self.delete_last_letter).pack(fill='x', pady=1)

        ttk.Button(edit_frame, text="⌫ Delete Word", 
                  command=self.delete_last_word).pack(fill='x', pady=1)

        # Text output area
        text_frame = tk.Frame(right_panel, bg=COLORS['surface'])
        text_frame.pack(fill='both', expand=True, padx=20, pady=(0, 20))

        tk.Label(text_frame, text="Text Output", 
                font=('Arial', 16, 'bold'), 
                fg=COLORS['text_primary'], 
                bg=COLORS['surface']).pack(anchor='w', pady=(0, 10))

        # Current word
        word_frame = tk.Frame(text_frame, bg=COLORS['background'], relief='flat', bd=1)
        word_frame.pack(fill='x', pady=(0, 10))

        tk.Label(word_frame, text="Current Word:", 
                font=('Arial', 10, 'bold'), 
                fg=COLORS['text_secondary'], 
                bg=COLORS['background']).pack(anchor='w', padx=10, pady=(5, 0))

        self.word_display = tk.Label(word_frame, 
                                    text="", 
                                    font=('Arial', 14),
                                    fg=COLORS['text_primary'], 
                                    bg=COLORS['background'],
                                    wraplength=350,
                                    justify='left')
        self.word_display.pack(anchor='w', padx=10, pady=(0, 10))

        # Sentence area
        sentence_frame = tk.Frame(text_frame, bg=COLORS['background'], relief='flat', bd=1)
        sentence_frame.pack(fill='both', expand=True)

        tk.Label(sentence_frame, text="Sentence:", 
                font=('Arial', 10, 'bold'), 
                fg=COLORS['text_secondary'], 
                bg=COLORS['background']).pack(anchor='w', padx=10, pady=(5, 0))

        # Scrollable text area
        text_scroll_frame = tk.Frame(sentence_frame, bg=COLORS['background'])
        text_scroll_frame.pack(fill='both', expand=True, padx=10, pady=(0, 10))

        self.sentence_text = tk.Text(text_scroll_frame, 
                                    font=('Arial', 12),
                                    fg=COLORS['text_primary'],
                                    bg=COLORS['background'],
                                    relief='flat',
                                    wrap='word',
                                    height=8)
        
        scrollbar = ttk.Scrollbar(text_scroll_frame, orient='vertical', command=self.sentence_text.yview)
        self.sentence_text.configure(yscrollcommand=scrollbar.set)
        
        self.sentence_text.pack(side='left', fill='both', expand=True)
        scrollbar.pack(side='right', fill='y')

        # Keyboard shortcuts info
        shortcuts_frame = tk.Frame(right_panel, bg=COLORS['surface'])
        shortcuts_frame.pack(fill='x', padx=20, pady=(0, 20))

        tk.Label(shortcuts_frame, text="Keyboard Shortcuts", 
                font=('Arial', 10, 'bold'), 
                fg=COLORS['text_secondary'], 
                bg=COLORS['surface']).pack(anchor='w')

        shortcuts_text = """Ctrl+S: Start Camera
Ctrl+Q: Stop Camera
Ctrl+C: Clear All
Ctrl+Space: Speak Text
Backspace: Delete Letter
Ctrl+Backspace: Delete Word"""

        tk.Label(shortcuts_frame, text=shortcuts_text, 
                font=('Arial', 8), 
                fg=COLORS['text_secondary'], 
                bg=COLORS['surface'],
                justify='left').pack(anchor='w', pady=(5, 0))

    def create_status_bar(self):
        """Create the status bar at the bottom"""
        self.status_bar = tk.Frame(self.root, bg=COLORS['border'], height=30)
        self.status_bar.pack(fill='x', side='bottom')
        self.status_bar.pack_propagate(False)

        self.status_text = tk.Label(self.status_bar, 
                                   text="Ready to start sign language recognition...", 
                                   font=('Arial', 10),
                                   fg=COLORS['text_secondary'], 
                                   bg=COLORS['border'])
        self.status_text.pack(side='left', padx=10, pady=5)

        # Time display
        self.time_label = tk.Label(self.status_bar, 
                                  text="", 
                                  font=('Arial', 10),
                                  fg=COLORS['text_secondary'], 
                                  bg=COLORS['border'])
        self.time_label.pack(side='right', padx=10, pady=5)
        self.update_time()

    def update_time(self):
        """Update the time display"""
        current_time = datetime.now().strftime("%H:%M:%S")
        self.time_label.config(text=current_time)
        self.root.after(1000, self.update_time)

    def update_status(self, message):
        """Update the status bar message"""
        self.status_text.config(text=message)

    # === Button Functionalities ===
    def start_video(self):
        if not self.video_running:
            try:
                self.cap = cv2.VideoCapture(0)
                if not self.cap.isOpened():
                    messagebox.showerror("Error", "Could not open camera")
                    return
                
                self.video_running = True
                self.camera_status.config(text="● Camera: ON", fg=COLORS['success'])
                self.start_btn.config(state='disabled')
                self.stop_btn.config(state='normal')
                self.update_status("Camera started. Show hand gestures to begin recognition.")
                self.update_video()
            except Exception as e:
                messagebox.showerror("Error", f"Failed to start camera: {str(e)}")

    def stop_video(self):
        self.video_running = False
        if self.cap:
            self.cap.release()
        self.video_label.config(image='', text="📹 Camera Feed\n\nPress 'Start Camera' to begin")
        self.camera_status.config(text="● Camera: OFF", fg=COLORS['danger'])
        self.start_btn.config(state='normal')
        self.stop_btn.config(state='disabled')
        self.letter_display.config(text="—")
        self.confidence_var.set(0)
        self.update_status("Camera stopped.")

    def clear_predictions(self):
        self.word = ""
        self.sentence = ""
        self.saved_sentences.clear()
        self.word_display.config(text="")
        self.sentence_text.delete('1.0', tk.END)
        self.letter_display.config(text="—")
        self.confidence_var.set(0)
        self.update_status("All text cleared.")

    def speak_text(self):
        full_sentence = (self.sentence + self.word).strip()
        if full_sentence:
            try:
                self.update_status("Speaking text...")
                result = synthesizer.speak_text_async(full_sentence).get()
                if result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
                    self.update_status("Speech synthesis completed.")
                else:
                    self.update_status("Speech synthesis failed.")
            except Exception as e:
                messagebox.showerror("Speech Error", f"Azure speech error: {str(e)}")
                self.update_status("Speech synthesis failed.")

            self.saved_sentences.append(full_sentence)
            self.sentence_text.insert(tk.END, full_sentence + "\n")
            self.sentence_text.see(tk.END)
            self.word = ""
            self.sentence = ""
            self.word_display.config(text="")
        else:
            messagebox.showwarning("Warning", "No text to speak!")

    def delete_last_letter(self):
        if self.word:
            self.word = self.word[:-1]
            self.word_display.config(text=self.word)
            self.update_status("Deleted last letter.")
        elif self.sentence:
            self.sentence = self.sentence.rstrip()
            if self.sentence and self.sentence[-1] == " ":
                self.sentence = self.sentence[:-1]
            self.word = self.sentence.split()[-1] if self.sentence else ""
            self.sentence = " ".join(self.sentence.split()[:-1]) + " "
            self.word_display.config(text=self.word)
            self.update_status("Deleted last letter.")

    def delete_last_word(self):
        if self.word:
            self.word = ""
            self.word_display.config(text="")
            self.update_status("Deleted current word.")
        elif self.sentence:
            self.sentence = self.sentence.rstrip()
            words = self.sentence.split()
            self.sentence = " ".join(words[:-1]) + " " if words else ""
            self.update_status("Deleted last word.")

    def update_video(self):
        if not self.video_running:
            return

        ret, frame = self.cap.read()
        if not ret:
            self.stop_video()
            return

        frame = cv2.flip(frame, 1)
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb)

        current_time = time.time()
        current_letter = ""
        confidence = 0.0

        if results.multi_hand_landmarks:
            self.last_seen_time = current_time
            for hand_landmarks in results.multi_hand_landmarks:
                # Draw landmarks with custom styling
                mp_drawing.draw_landmarks(
                    frame, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(255, 255, 0), thickness=2))
                
                landmarks = [(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark]

                try:
                    if model and le:
                        X = normalize_landmarks(landmarks)
                        pred_proba = model.predict_proba(X)[0]
                        pred_index = np.argmax(pred_proba)
                        confidence = pred_proba[pred_index] * 100
                        current_letter = le.inverse_transform([pred_index])[0]

                        if current_letter == self.prev_letter:
                            if not self.letter_hold_start:
                                self.letter_hold_start = current_time
                            if current_time - self.letter_hold_start >= 1:
                                if not self.word or self.word[-1] != current_letter:
                                    self.word += current_letter
                                    self.word_display.config(text=self.word)
                                    self.update_status(f"Added letter: {current_letter}")
                        else:
                            self.letter_hold_start = current_time

                        self.prev_letter = current_letter
                    else:
                        current_letter = "X"  # Demo mode
                        confidence = 85.0

                except Exception as e:
                    print("Prediction error:", e)
                    self.update_status("Prediction error occurred.")
        else:
            time_since_last = current_time - self.last_seen_time
            if time_since_last >= 2 and self.word and (not self.word.endswith(" ")):
                self.word += " "
                self.word_display.config(text=self.word)
            if time_since_last >= 5 and self.word.strip():
                self.sentence += self.word.strip() + " "
                self.word = ""
                self.word_display.config(text="")
                self.update_status("Word added to sentence.")

        # Update displays
        self.letter_display.config(text=current_letter if current_letter else "—")
        self.confidence_var.set(confidence)

        # Update video feed
        # Resize frame to fit display
        height, width = frame.shape[:2]
        display_width = 640
        display_height = int(height * (display_width / width))
        frame_resized = cv2.resize(frame, (display_width, display_height))
        
        img = Image.fromarray(cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB))
        imgtk = ImageTk.PhotoImage(image=img)
        self.video_label.imgtk = imgtk
        self.video_label.configure(image=imgtk, text="")

        self.root.after(10, self.update_video)

# === Run the Application ===
if __name__ == "__main__":
    root = tk.Tk()
    app = ModernTSLApp(root)
    root.mainloop()

d:\WIN AND LEARN\.venv\Lib\site-packages\sklearn\base.py:440: InconsistentVersionWarning: Trying to unpickle estimator LabelBinarizer from version 1.6.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
d:\WIN AND LEARN\.venv\Lib\site-packages\sklearn\base.py:440: InconsistentVersionWarning: Trying to unpickle estimator MLPClassifier from version 1.6.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
d:\WIN AND LEARN\.venv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPro